In [17]:
%pip install pandas,numpy
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'pandas,numpy': Expected end or semicolon (after name and no valid version specifier)
    pandas,numpy
          ^


Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)

   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 

In [1]:
import pandas as pd
df = pd.read_csv('C:\\Users\\acer\\OneDrive\\Desktop\\New folder\\New folder (3)\\uk_sanctions_data.csv')
df = pd.DataFrame(df)

In [2]:
missing_value = df.isnull().sum()
print("Missing value per column",missing_value)

Missing value per column Name                1
Alias            1275
Nationality         0
Designation      1385
Sanction Type       0
Source              0
dtype: int64


In [3]:
column = 'Designation'
missing_percent = df[column].isnull().mean()*100
print(f"Missing percentage in '{column}' is {missing_percent:.2f}%")

Missing percentage in 'Designation' is 54.87%


In [4]:
train_df = df[df['Designation'].notnull()]
test_df = df[df['Designation'].isnull()]

In [11]:
df['input_text'] = df['Name']  + ' ' + df['Sanction Type']


In [ ]:
# Fill missing values in 'Name' and 'Nationality' if necessary (although you've indicated these are not the cause)
df['Name'] = df['Name'].fillna('Unknown')
df['Nationality'] = df['Nationality'].fillna('Unknown')

# Create 'input_text' by combining 'Name' and 'Nationality' (this will not contain any NaNs if handled correctly)
df['input_text'] = df['Name'] + ' ' + df['Nationality']

# Ensure that 'input_text' does not contain any NaNs (even though it shouldn't after combining 'Name' and 'Nationality')
df['input_text'] = df['input_text'].fillna('Unknown')

# Split the known data (those rows with a non-null 'Designation')
known_df = df[df['Designation'].notnull()]
X = known_df['input_text']
y = known_df['Designation']

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

# Train the Logistic Regression model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# Predict the Designation for the validation set
y_pred = clf.predict(X_val_vec)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

# Predict missing Designations (where 'Designation' is NaN)
X_test = df[df['Designation'].isnull()]['input_text']
X_test_vec = vectorizer.transform(X_test)  # Vectorize the text data for prediction
predicted_designations = clf.predict(X_test_vec)

# Fill missing Designations in the original dataframe
df.loc[df['Designation'].isnull(), 'Designation'] = predicted_designations

# View the dataframe with filled Designations
print(df.head())


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataFrame 'df' with 'Name', 'Nationality', and 'Designation'

# Fill missing values in 'Name' and 'Nationality'
df['Name'] = df['Name'].fillna('Unknown')
df['Nationality'] = df['Nationality'].fillna('Unknown')

# Create 'input_text' by combining 'Name' and 'Nationality'
df['input_text'] = df['Name'] + ' ' + df['Nationality']
df['input_text'] = df['input_text'].fillna('Unknown')  # Ensure no NaNs in input_text

# Split the known data (those rows with a non-null 'Designation')
known_df = df[df['Designation'].notnull()]
X = known_df['input_text']
y = known_df['Designation']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

# Train the Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# Predict the Designation for the validation set
y_pred = clf.predict(X_val_vec)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))


Accuracy: 0.695049504950495
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                                                                               (1) Investigator (2) Deputy Head of the 534th Military Investigation Department

C:\Users\acer\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(res

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Assuming you have a DataFrame 'df' with 'Name', 'Nationality', and 'Designation'

# Fill missing values in 'Name' and 'Nationality'
df['Name'] = df['Name'].fillna('Unknown')
df['Nationality'] = df['Nationality'].fillna('Unknown')

# Create 'input_text' by combining 'Name' and 'Nationality'
df['input_text'] = df['Name'] + ' ' + df['Nationality']
df['input_text'] = df['input_text'].fillna('Unknown')  # Ensure no NaNs in input_text

# Split the known data (those rows with a non-null 'Designation')
known_df = df[df['Designation'].notnull()]
X = known_df['input_text']
y = known_df['Designation']

# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_vec = vectorizer.fit_transform(X)  # Fit and transform the training data

# Train the Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_vec, y)

# Check if there are any missing 'Designation' values
if df['Designation'].isnull().sum() > 0:
    # Predict missing Designations (where 'Designation' is NaN)
    X_test = df[df['Designation'].isnull()]['input_text']  # Extract rows with missing 'Designation'
    
    if len(X_test) > 0:  # Ensure that there are rows with missing Designations
        X_test_vec = vectorizer.transform(X_test)  # Transform the test data with the fitted vectorizer
        predicted_designations = clf.predict(X_test_vec)

        # Fill the missing Designations with the predicted values
        df.loc[df['Designation'].isnull(), 'Designation'] = predicted_designations

        # Print the predicted Designations
        for idx, pred in enumerate(predicted_designations):
            print(f"Row {df[df['Designation'].isnull()].index[idx]} - Predicted Designation: {pred}")
    else:
        print("No missing Designations to predict.")
else:
    print("There are no missing Designations in the data.")


df['Nationality']
missing_value = df.isnull().sum()
print("Missing value per column",missing_value)



There are no missing Designations in the data.
Missing value per column Name                0
Alias            1275
Nationality         0
Designation         0
Sanction Type       0
Source              0
input_text          0
dtype: int64


In [32]:
# Save the modified DataFrame without 'input_text' to a new CSV file
df.to_csv('designation_predic.csv', index=False)


In [5]:
import pandas as pd
df = pd.read_csv('C:\\Users\\acer\\OneDrive\\Desktop\\New folder\\New folder (3)\\usofac-sdn_sanctions_data.csv')
df = pd.DataFrame(df)

In [6]:
column = 'Designation'
missing_percent = df[column].isnull().mean()*100
print(f"Missing percentage in '{column}' is {missing_percent:.2f}%")

Missing percentage in 'Designation' is 0.00%
